# Google Drive Tutorial
***

코드를 실행해보기 위해선 다음의 파이썬 라이브러리들이 설치되어있어야 한다.  
- google-api-python-client
- google-auth-oauthlib

Anaconda Prompt에서 다음과 같은 명령어로 설치한다.

```sh
pip install google-api-python-client
pip install google-auth-oauthlib
```

# 1. Google Drive API 사용설정
***


## 1.1. 프로젝트 선택 또는 새 프로젝트 생성
***

[Google API Console](https://console.developers.google.com/)에서 Google Drive API를 사용할 프로젝트를 선택한다.  
프로젝트가 없다면 다음과 같이 새 프로젝트를 만든다.

![새 프로젝트 생성](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/mk_project.gif)


## 1.2. Google Drive API 사용설정
***

![API 사용설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/enable_gd_api.gif)


## 1.3. OAuth 동의 화면 설정
***

![OAuth 동의 화면 설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/oauth_consent_screen.gif)


## 1.4. 사용자 인증 정보 생성
***

![사용자 인증 정보 생성](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/mk_oauth_key.gif)


## 1.5. 공통적으로 사용하는 코드
***

build()로 리턴되는 drive_service는 <https://developers.google.com/resources/api-libraries/documentation/drive/v3/python/latest>를 참조한다.

In [1]:
# -*- coding: utf-8 -*-
import io
import pprint
import pickle
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# 다운 받은 키 파일명을 credentials.json로 변경하거나 아래 코드를 변경
CLIENT_SECRET_FILE_NAME = "client_secret.json"
TOKEN_FILE_NAME = "token.pickle"

# SCOPES에 들어가는 값들은 다음 URL을 참조: https://developers.google.com/drive/api/v3/about-auth
SCOPES = [
    "https://www.googleapis.com/auth/drive"
]

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists(TOKEN_FILE_NAME):
    with open(TOKEN_FILE_NAME, "rb") as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            CLIENT_SECRET_FILE_NAME, SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open(TOKEN_FILE_NAME, "wb") as token:
        pickle.dump(creds, token)

drive_service = build("drive", "v3", credentials=creds)

# 2. 기본적인 사용
***


## 2.1. 드라이브 정보
***

fields에 들어가는 인자들은 <https://developers.google.com/drive/api/v3/reference/about>를 참조한다.

In [2]:
response = drive_service.about().get(fields="kind, storageQuota").execute()

pprint.pprint(response)

{'kind': 'drive#about',
 'storageQuota': {'usage': '87125928048',
                  'usageInDrive': '87125928048',
                  'usageInDriveTrash': '5503'}}


## 2.2. 읽기
***


### 2.2.1. 특정 파일/폴더의 ID를 알아내기
***

![파일/폴더의 ID 알아내기](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/get_file_id.gif)



### 2.2.2. 파일/폴더 검색
***

query를 이용하여 원하는 파일 또는 폴더를 검색할 수 있다.  
참조: <https://developers.google.com/drive/api/v3/search-parameters>  
다음은 특정 폴더에 있는 test1.txt라는 파일을 찾는 예제이다.

In [3]:
FOLDER_ID = "1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4"

response = drive_service.files().list(
    q="'{0}' in parents and name = '{1}'".format(FOLDER_ID, "test1.txt"),
    spaces="drive",
    fields="files(id, name, mimeType, size)"
).execute()

files = response.get("files", [])

print("\n##### Metadata #####\n")
print(files)


##### Metadata #####

[{'id': '1Gh1Xdgp9FLFfsf_20Ou_V_mULhz9ZXk5', 'name': 'test1.txt', 'mimeType': 'text/plain', 'size': '3'}]


### 2.2.3. Paging
***
폴더내의 목록을 불러올 때 기본적으로 100개까지 불러오게 되며, pageSize에 값을 줘서 한 번에 조회되는 개수를 조절할 수 있다. (1 ~ 1000개)  
조회하려는 목록이 한 번에 조회되는 목록보다 많을 때는 Paging 처리를 해야 한다.

In [4]:
FOLDER_ID = "17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ"
page_token = None

while True:
    print("\n##### List of Metadata #####\n")

    response = drive_service.files().list(
        q="'{0}' in parents".format(FOLDER_ID),
        spaces="drive",
        fields="nextPageToken, files(id, name, mimeType, parents)",
        pageSize=3,  # 한 번에 조회할 개수
        pageToken=page_token,
        orderBy="name",
    ).execute()

    files = response.get("files", [])
    pprint.pprint(files)

    print("\n##### nextPageToken #####\n")

    page_token = response.get('nextPageToken', None)
    print(page_token)

    if page_token is None:
        break;


##### List of Metadata #####

[{'id': '1hlfZu-LmjCYKduwE7ft8keoEQi_6_t4t',
  'mimeType': 'text/plain',
  'name': 'sub_file_1.txt',
  'parents': ['17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ']},
 {'id': '1gmtqZvJsNku3pbw2XIDFxP2arc_vMaH7',
  'mimeType': 'text/plain',
  'name': 'sub_file_2.txt',
  'parents': ['17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ']},
 {'id': '1Ei3-514r28E1HYAMYxrI7aE9l2adT1hX',
  'mimeType': 'text/plain',
  'name': 'sub_file_3.txt',
  'parents': ['17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ']}]

##### nextPageToken #####

~!!~AI9FV7T59la2smTVqJH4x-lqS0AOLqo5pY4tXG1PeRuKQMceOMFKU4LIxZfkBLIcyfxYMlxFjDReagWwhI_VYeP45ecbwGYk2IK7r7hPl92FXUDDMORy69ThlelKNRDIkB4vP01xcRPT2bZivEjptPvjHiESTHXfzkh5z4BaW3jzP1gafAdvA-hQVOjbuDlw433ZcY4QugsNR3AWvcfuhMX7AOmYjDKacZGXljQHdkj0_jeeSoLwph7JVKA1cLajBX43FYoUqQwXBZRJYdYi5I2hRDY8C2jiFVf9yH5eOsD3CiM1MbElkkOHMj0Y5dfluP_i9PYvhdncq7dD30UL_aqJSJ0zFCTKjGHBVqBGbZbjjG-PT5s1DsE=

##### List of Metadata #####

[{'id': '1BJyvc14JilzVckwU4I6I4IeLaH8FiWgl',
  'mimeType': 'text/pl

### 2.2.4. 폴더/파일의 ID를 이용하여 Metadata를 출력
***

In [5]:
FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 샘플로 읽어볼 파일의 ID
FOLDER_ID = "1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4"  # 샘플로 읽어볼 폴더의 ID

# fields에 들어가는 인자는 다음 URL을 참조: https://developers.google.com/drive/api/v3/reference/files
FIELDS = "kind, id, name, mimeType, size"

print("\n##### File's MetaData #####\n")
response = drive_service.files().get(fileId=FILE_ID, fields=FIELDS).execute()
pprint.pprint(response)

print("\n\n##### Folder's MetaData #####\n")
response = drive_service.files().get(fileId=FOLDER_ID, fields=FIELDS).execute()
pprint.pprint(response)


##### File's MetaData #####

{'id': '1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5',
 'kind': 'drive#file',
 'mimeType': 'text/plain',
 'name': 'Test.txt',
 'size': '10'}


##### Folder's MetaData #####

{'id': '1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4',
 'kind': 'drive#file',
 'mimeType': 'application/vnd.google-apps.folder',
 'name': 'sample1'}


### 2.2.5. Batch Requests
***

In [6]:
FILE_IDS = [
    "1W9tRlK7zu74Iyls2ijHkNkhVo_ftcQBQ",
    "1RN1rbdGfAlZ0tC87Vxib8c89ySgV2xvj",
    "12XmKiypLZnuQKpk_-g8wyu_vvQ0QMQxa",
]

FIELDS = "id, name, mimeType"

def callback(request_id, response, exception):
    if exception:
        # Handle error
        print("\n##### Exception #####\n")
        print(exception)
    else:
        print("\n##### Result #####\n")
        print("Request ID: {0}".format(request_id))
        print("Response: {0}".format(response))

batch = drive_service.new_batch_http_request(callback=callback)
batch.add(drive_service.files().get(fileId=FILE_IDS[0], fields=FIELDS))
batch.add(drive_service.files().get(fileId=FILE_IDS[1], fields=FIELDS))
batch.add(drive_service.files().get(fileId=FILE_IDS[2], fields=FIELDS))
batch.execute()


##### Result #####

Request ID: 1
Response: {'id': '1W9tRlK7zu74Iyls2ijHkNkhVo_ftcQBQ', 'name': 'sub_file2_1.txt', 'mimeType': 'text/plain'}

##### Result #####

Request ID: 2
Response: {'id': '1RN1rbdGfAlZ0tC87Vxib8c89ySgV2xvj', 'name': 'sub_file2_2.txt', 'mimeType': 'text/plain'}

##### Result #####

Request ID: 3
Response: {'id': '12XmKiypLZnuQKpk_-g8wyu_vvQ0QMQxa', 'name': 'sub_file2_3.txt', 'mimeType': 'text/plain'}


### 2.2.6. 파일 다운로드
***

In [7]:
from googleapiclient.http import MediaIoBaseDownload

FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 다운로드 받을 파일의 ID

print("\n##### Download Start #####\n")
request = drive_service.files().get_media(fileId=FILE_ID)
buffer = io.BytesIO()
downloader = MediaIoBaseDownload(buffer, request)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

print("\n\n##### Download Result #####\n")
print(buffer.getvalue())


##### Download Start #####

Download 100%.


##### Download Result #####

b'Test Body~'


## 2.3. 쓰기
***

### 2.3.1. 파일을 읽어서 업로드
***

In [8]:
from googleapiclient.http import MediaFileUpload

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

meta_data = {
    "name": "file_up_test.txt",
    "parents": [FOLDER_ID],
}

uploader = MediaFileUpload("upload_test.txt", mimetype="text/plain")
response = drive_service.files().create(body=meta_data, media_body=uploader).execute()

print("\n\n##### Upload Result #####\n")
print(response)



##### Upload Result #####

{'kind': 'drive#file', 'id': '1lEJXJ-5PynyzbmRZCROfLZB9B328NUd_', 'name': 'file_up_test.txt', 'mimeType': 'text/plain'}


### 2.3.2. 메모리에 있는걸 업로드
***

In [9]:
from googleapiclient.http import MediaIoBaseUpload

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

meta_data = {
    "name": "mem_up_test.txt",
    "parents": [FOLDER_ID],
}

buffer = io.BytesIO()
buffer.write(b"hello world!")

uploader = MediaIoBaseUpload(fd=buffer, mimetype="text/plain")
response = drive_service.files().create(
    body=meta_data, 
    media_body=uploader
).execute()

print("\n\n##### Upload Result #####\n")
print(response)



##### Upload Result #####

{'kind': 'drive#file', 'id': '1vB8u3s75MvczXhtqxGJS7obKUTwp-aUD', 'name': 'mem_up_test.txt', 'mimeType': 'text/plain'}


## 2.4. 수정
***

In [10]:
FILE_ID = response["id"]  # 위에서 메모리 업로드로 Create된 파일의 ID

meta_data = {
    "name": "mem_up_test2.txt",
}

buffer = io.BytesIO()
buffer.write(b"hi")

uploader = MediaIoBaseUpload(fd=buffer, mimetype="text/plain")
response = drive_service.files().update(
    fileId=FILE_ID,
    body=meta_data, 
    media_body=uploader
).execute()

print("\n##### Upload Result #####\n")
print(response)


##### Upload Result #####

{'kind': 'drive#file', 'id': '1vB8u3s75MvczXhtqxGJS7obKUTwp-aUD', 'name': 'mem_up_test2.txt', 'mimeType': 'text/plain'}


## 2.5. 삭제
***

In [11]:
response = drive_service.files().delete(fileId=FILE_ID).execute()

print("\n##### Delete Result #####\n")

if 0 == len(response):
    print("Complete")
else:
    print("Failed to delete")


##### Delete Result #####

Complete


# 3. 응용
***


## 3.1. 사용할 데이터를 바이너리 형태로 업로드
***

In [12]:
import struct

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

df = pd.read_csv("day_005930.csv")
print("\n##### Raw Data #####\n")
print(df)

buffer = io.BytesIO()

for _, row in df.iterrows():
    buffer.write(struct.pack("iiiiii", row["date"], row["open"], row["high"], row["low"], row["close"], row["volume"]))

print("\n##### Binary Data #####\n")
print(buffer.getvalue())

print("\n##### Length of Binary Data #####\n")
print(len(buffer.getvalue()))

meta_data = {
    "name": "day_005930.bry",
    "parents": [FOLDER_ID],
}

uploader = MediaIoBaseUpload(fd=buffer, mimetype="application/octet-stream")
response = drive_service.files().create(
    body=meta_data, 
    media_body=uploader
).execute()

print("\n\n##### Upload Result #####\n")
pprint.pprint(response)


##### Raw Data #####

       date   open   high    low  close    volume
0  20190329  44500  44900  44200  44650  11491713
1  20190401  45200  45450  44850  45050   7362129
2  20190402  45550  46100  45350  45750   9480688
3  20190403  46750  46750  45800  46600  12436570
4  20190404  46150  47100  46150  46950  12650168
5  20190405  46950  47550  46600  46850   8546339
6  20190408  47250  47250  46150  46650   8507909
7  20190409  46700  46950  46200  46650   6878761
8  20190410  46400  46700  46050  46700  11883995
9  20190411  46700  46800  46150  46250  13697399

##### Binary Data #####

b'y\x144\x01\xd4\xad\x00\x00d\xaf\x00\x00\xa8\xac\x00\x00j\xae\x00\x00\x81Y\xaf\x00\xc1\x144\x01\x90\xb0\x00\x00\x8a\xb1\x00\x002\xaf\x00\x00\xfa\xaf\x00\x00QVp\x00\xc2\x144\x01\xee\xb1\x00\x00\x14\xb4\x00\x00&\xb1\x00\x00\xb6\xb2\x00\x00\xf0\xa9\x90\x00\xc3\x144\x01\x9e\xb6\x00\x00\x9e\xb6\x00\x00\xe8\xb2\x00\x00\x08\xb6\x00\x00Z\xc4\xbd\x00\xc4\x144\x01F\xb4\x00\x00\xfc\xb7\x00\x00F\xb4\x00\x00f\

## 3.2. 데이터 다운로드
***

### 3.2.1 특정 지점부터 다운로드
***

In [13]:
FILE_ID = response["id"]  # 다운로드 받을 파일의 ID
START_READING = 6  # 다운받기 시작할 Row Index
STRUCT_SIZE = 24

print("\n##### Download Start #####\n")
request = drive_service.files().get_media(fileId=FILE_ID)
buffer = io.BytesIO()
downloader = MediaIoBaseDownload(buffer, request)
downloader._progress = STRUCT_SIZE * (START_READING - 1)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

print("\n\n##### Download Result #####\n")

rcv_data = buffer.getvalue()
st_idx = 0
rcv_len = len(rcv_data)

while rcv_len > st_idx:
    ed_idx = st_idx + STRUCT_SIZE
    print(struct.unpack("iiiiii", rcv_data[st_idx:ed_idx]))
    st_idx = ed_idx


##### Download Start #####

Download 100%.


##### Download Result #####

(20190405, 46950, 47550, 46600, 46850, 8546339)
(20190408, 47250, 47250, 46150, 46650, 8507909)
(20190409, 46700, 46950, 46200, 46650, 6878761)
(20190410, 46400, 46700, 46050, 46700, 11883995)
(20190411, 46700, 46800, 46150, 46250, 13697399)


### 3.2.2. 스트리밍
***

In [14]:
STRUCT_SIZE = 24
CHUNK_SIZE = STRUCT_SIZE * 3


class BufferHandler(io.BytesIO):
    def __init__(self):
        super(BufferHandler, self).__init__()
        self.buffer = None

    def write(self, data):
        self.buffer = data
        super(BufferHandler, self).write(data)

        
print("\n##### Download Start #####\n")
request = drive_service.files().get_media(fileId=FILE_ID)
buffer_handler = BufferHandler()
downloader = MediaIoBaseDownload(buffer_handler, request, chunksize=CHUNK_SIZE - 1)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print(buffer_handler.buffer)
    print("")
    
    st_idx = 0
    rcv_len = len(buffer_handler.buffer)
    
    while CHUNK_SIZE > st_idx and rcv_len > st_idx:
        
        ed_idx = st_idx + STRUCT_SIZE
        rcv_data = struct.unpack("iiiiii", buffer_handler.buffer[st_idx:ed_idx])
        print(rcv_data)
        st_idx = ed_idx
        
    print("\nDownload %d%%\n" % int(status.progress() * 100))

print("##### Done #####\n")


##### Download Start #####

b'y\x144\x01\xd4\xad\x00\x00d\xaf\x00\x00\xa8\xac\x00\x00j\xae\x00\x00\x81Y\xaf\x00\xc1\x144\x01\x90\xb0\x00\x00\x8a\xb1\x00\x002\xaf\x00\x00\xfa\xaf\x00\x00QVp\x00\xc2\x144\x01\xee\xb1\x00\x00\x14\xb4\x00\x00&\xb1\x00\x00\xb6\xb2\x00\x00\xf0\xa9\x90\x00'

(20190329, 44500, 44900, 44200, 44650, 11491713)
(20190401, 45200, 45450, 44850, 45050, 7362129)
(20190402, 45550, 46100, 45350, 45750, 9480688)

Download 30%

b'\xc3\x144\x01\x9e\xb6\x00\x00\x9e\xb6\x00\x00\xe8\xb2\x00\x00\x08\xb6\x00\x00Z\xc4\xbd\x00\xc4\x144\x01F\xb4\x00\x00\xfc\xb7\x00\x00F\xb4\x00\x00f\xb7\x00\x00\xb8\x06\xc1\x00\xc5\x144\x01f\xb7\x00\x00\xbe\xb9\x00\x00\x08\xb6\x00\x00\x02\xb7\x00\x00#h\x82\x00'

(20190403, 46750, 46750, 45800, 46600, 12436570)
(20190404, 46150, 47100, 46150, 46950, 12650168)
(20190405, 46950, 47550, 46600, 46850, 8546339)

Download 60%

b'\xc8\x144\x01\x92\xb8\x00\x00\x92\xb8\x00\x00F\xb4\x00\x00:\xb6\x00\x00\x05\xd2\x81\x00\xc9\x144\x01l\xb6\x00\x00f\xb7\x00\x00x\

# 4. Google Sheets
***


## 4.1. Google Sheets API 사용설정
***

![Google Sheets API 사용설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/enable_google_sheets.gif)


## 4.2. Sheets Service 객체 생성
***

생성되는 sheets_service는 <https://developers.google.com/resources/api-libraries/documentation/sheets/v4/python/latest/>를 참조한다.

In [15]:
sheets_service = build("sheets", "v4", credentials=creds)

## 4.3. 읽기
***


### 4.3.1. 파일 정보 얻기
***

In [16]:
SPREAD_SHEET_ID = "1PH9fjNRyKd0IqBiaZ3cDGhysmP3Y2241CHCEdt2nQrc"

sheet_obj = sheets_service.spreadsheets()
response = sheet_obj.get(spreadsheetId=SPREAD_SHEET_ID).execute()
pprint.pprint(response)

{'properties': {'autoRecalc': 'ON_CHANGE',
                'defaultFormat': {'backgroundColor': {'blue': 1,
                                                      'green': 1,
                                                      'red': 1},
                                  'padding': {'bottom': 2,
                                              'left': 3,
                                              'right': 3,
                                              'top': 2},
                                  'textFormat': {'bold': False,
                                                 'fontFamily': 'arial,sans,sans-serif',
                                                 'fontSize': 10,
                                                 'foregroundColor': {},
                                                 'italic': False,
                                                 'strikethrough': False,
                                                 'underline': False},
                                

### 4.3.2. 특정 범위 읽기
***

다른 사용법은 <https://developers.google.com/sheets/api/samples/reading>를 참조한다.  
RANGE에 들어갈 A1 Notation은 <https://developers.google.com/sheets/api/guides/concepts#a1_notation>를 참조

In [17]:
SPREAD_SHEET_ID = "1PH9fjNRyKd0IqBiaZ3cDGhysmP3Y2241CHCEdt2nQrc"
SHEET_NAME = response["sheets"][0]["properties"]["title"]

rng = "{0}!A:D".format(SHEET_NAME)
spreadsheet_obj = sheets_service.spreadsheets()
response = spreadsheet_obj.values().get(spreadsheetId=SPREAD_SHEET_ID, range=rng).execute()
print("\n##### Range A:D #####\n")
pprint.pprint(response)

rng = [
    "{0}!A1:D1".format(SHEET_NAME),
    "{0}!A5:D8".format(SHEET_NAME),
]
spreadsheet_obj = sheets_service.spreadsheets()
response = spreadsheet_obj.values().batchGet(spreadsheetId=SPREAD_SHEET_ID, ranges=rng).execute()
print("\n##### Range A1:D1 And A5:D8 #####\n")
pprint.pprint(response)


##### Range A:D #####

{'majorDimension': 'ROWS',
 'range': "'시트1'!A1:D1000",
 'values': [['market', 'fullcode', 'shtcode', 'name'],
            ['kospi', 'KR7024890006', 'A024890', '대원화성'],
            ['kospi', 'KR7024900003', 'A024900', '덕양산업'],
            ['kosdaq', 'KR7024910002', 'A024910', '경창산업'],
            ['kosdaq', 'KR7024940009', 'A024940', 'PN풍년'],
            ['kosdaq', 'KR7024950008', 'A024950', '삼천리자전거'],
            ['kospi', 'KR7025000001', 'A025000', 'KPX케미칼'],
            ['kosdaq', 'KR7025320003', 'A025320', '시노펙스'],
            ['kosdaq', 'KR7025440009', 'A025440', '대성엘텍'],
            ['kospi', 'KR7025530007', 'A025530', 'SJM홀딩스'],
            ['kospi', 'KR7025540006', 'A025540', '한국단자'],
            ['kosdaq', 'KR7025550005', 'A025550', '한국선재'],
            ['kospi', 'KR7025560004', 'A025560', '미래산업'],
            ['kospi', 'KR7025750001', 'A025750', '한솔홈데코'],
            ['kosdaq', 'KR7025770009', 'A025770', '한국정보통신'],
            ['kospi', 'KR7025820002', 

## 4.4. 쓰기
***


### 4.4.1. 특정 폴더에 Spreadsheet 파일을 생성
***

<https://developers.google.com/sheets/api/guides/create>에서 설명한 대로 Spreadsheet 파일을 만들 수도 있지만,  
특정한 폴더내에 파일을 생성하는 방법은 못찾았다.  
Google Drive API를 사용하여 파일을 생성하겠다.  
Spreadsheet의 MIME Type은 "application/vnd.google-apps.spreadsheet"이다,  
구글에서 지원하는 MIME Type는 <https://developers.google.com/drive/api/v3/mime-types>를 참조한다.

In [18]:
import datetime

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 생성
FILE_NAME = "codetbl_{0}".format(datetime.date.today().strftime("%Y%m%d"))

meta_data = {
    "name": FILE_NAME,
    "parents": [FOLDER_ID],
    "mimeType": "application/vnd.google-apps.spreadsheet",
}

response = drive_service.files().create(body=meta_data).execute()

print("\n\n##### Upload Result #####\n")
pprint.pprint(response)



##### Upload Result #####

{'id': '1NBjZgCzOcoI4OgqYEUhUV0seS5BL8RTKRYmb7em25nM',
 'kind': 'drive#file',
 'mimeType': 'application/vnd.google-apps.spreadsheet',
 'name': 'codetbl_20190412'}


### 4.4.2. 생성된 파일에 쓰기
***

다른 사용법은 <https://developers.google.com/sheets/api/samples/writing>를 참조한다.

In [19]:
SPREAD_SHEET_ID = response["id"]

df = pd.read_csv("code_info.csv")
print("\n##### Raw Data #####\n")
print(df)

input_values = [list(df)]
input_values.extend(df.values.tolist())

body_data = {
    "values": input_values
}

response = spreadsheet_obj.values().update(
    spreadsheetId=SPREAD_SHEET_ID, 
    range="시트1!A1:C5", 
    body=body_data, 
    valueInputOption="RAW"  # "RAW" or "USER_ENTERED"
).execute()

print("\n##### Update Result #####\n")
pprint.pprint(response)


##### Raw Data #####

       fullcode  shtcode       name
0  KR7005930003  A005930       삼성전자
1  KR7046890000  A046890      서울반도체
2  KR7069500007  A069500  KODEX 200
3  KR7114800006  A114800  KODEX 인버스

##### Update Result #####

{'spreadsheetId': '1NBjZgCzOcoI4OgqYEUhUV0seS5BL8RTKRYmb7em25nM',
 'updatedCells': 15,
 'updatedColumns': 3,
 'updatedRange': "'시트1'!A1:C5",
 'updatedRows': 5}


### 4.4.3. Append
***

In [20]:
body_data = {
    "values": [
        ["KR7035720002", "A035720", "카카오"],
        ["KR7084990001", "A084990", "바이로메드"]
    ]
}

response = spreadsheet_obj.values().append(
    spreadsheetId=SPREAD_SHEET_ID,
    range="시트1!A6:C6",
    body=body_data,
    valueInputOption="RAW",  # "RAW" or "USER_ENTERED"
).execute()

print("\n##### Append Result #####\n")
pprint.pprint(response)


##### Append Result #####

{'spreadsheetId': '1NBjZgCzOcoI4OgqYEUhUV0seS5BL8RTKRYmb7em25nM',
 'updates': {'spreadsheetId': '1NBjZgCzOcoI4OgqYEUhUV0seS5BL8RTKRYmb7em25nM',
             'updatedCells': 6,
             'updatedColumns': 3,
             'updatedRange': "'시트1'!A6:C7",
             'updatedRows': 2}}
